<a href="https://colab.research.google.com/github/trisha025/Image_Classification_TransferLearning/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip3 install torch
!pip3 install torchvision

In [0]:
from torchvision import transforms  

In [0]:
image_transforms = {
    'train' : transforms.Compose([
                                  transforms.RandomResizedCrop(size=256, scale=(0.8,1.0)),
                                  transforms.RandomRotation(degrees=15),
                                  transforms.CenterCrop(size=224),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406],
                                                       [0.229, 0.224, 0.225])
    ]),
    'valid' : transforms.Compose([
                                  transforms.Resize(size=256),
                                  transforms.CenterCrop(size=224),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406],
                                                       [0.229, 0.224, 0.225])
    ]),
    'test' : transforms.Compose([
                                 transforms.Resize(size=256),
                                 transforms.CenterCrop(size=224),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                      [0.229, 0.224, 0.225])
    ])
}